# Weitere Beispiele

Hier sind weitere Beispiele.

In [1]:
from manim import *

Manim Community v0.15.1

## Kathetensatz

Zunächst ein Beweis des Kathetensatzes. Man könnte noch den zweiten Teil hinzufügen.

In [2]:
%%manim -ql -v WARNING Kathetensatz

import numpy as np

# PARAMETERS
r = 2
angle = np.pi*2/5
rt = 3

# colours
colour_tri = "#5be800"
colour_quad = "#0777ba"

common_kwargs = {"fill_opacity": 0.5}

class Kathetensatz(Scene):
    def construct(self):
        # Parameters
        pa = np.array([-r, 0, 0])
        pb = np.array([r, 0, 0])
        pc = np.array([r*np.cos(angle), r*np.sin(angle), 0])
        a = np.sqrt((pc[0]-pb[0])**2 + pc[1]**2)
        triangle = Polygon(pa, pb, pc, color=colour_tri, **common_kwargs)
        self.add(triangle)

        t_tracker = ValueTracker(0)
        p1 = pb
        p2 = pb + a*np.array([np.cos(angle/2), np.sin(angle/2), 0])
        p3_0 = p2 + pc - pb
        p4_0 = pc
        square = Polygon(p1, p2, p3_0, p4_0, color=colour_quad, **common_kwargs)
        self.add(square)

        # a^2
        text_a2 = MathTex("a^2").shift(square.get_center())
        self.add(text_a2)
        text_eq = MathTex("a^2", "=", "pc").shift(1.5*r*UP)
        # animation updater
        self.play(text_a2.animate.move_to(text_eq.get_part_by_tex("a^2").get_center()))
        square.add_updater(
            lambda x: x.become(Polygon(p1, p2, p3_0 + (pa - pc)*t_tracker.get_value(), p4_0 + (pa - pc)*t_tracker.get_value(), color=colour_quad, **common_kwargs))
        )
        self.play(t_tracker.animate.increment_value(1), run_time=rt)

        angle_tracker = ValueTracker(0)
        p4 = pa
        p3 = p4 + p2-p1
        p3d = p3 - p1
        square.add_updater(
            lambda x: x.become(Polygon(pb, pb + a*np.array([np.cos(angle/2 + angle_tracker.get_value()), np.sin(angle/2 + angle_tracker.get_value()), 0]),
                                p1 + np.array([p3d[0]*np.cos(angle_tracker.get_value())-p3d[1]*np.sin(angle_tracker.get_value()),
                                 p3d[0]*np.sin(angle_tracker.get_value())+p3d[1]*np.cos(angle_tracker.get_value()),0]),
                                pb + 2*r*np.array([-np.cos(angle_tracker.get_value()), -np.sin(angle_tracker.get_value()), 0]),
                                color=colour_quad, **common_kwargs))
        )
        self.play(angle_tracker.animate.increment_value(np.pi/2), run_time=rt)
        

        p2 = pc
        p3 = pc + np.array([0, -2*r, 0])
        p4 = p1 + np.array([0, -2*r, 0])

        s_tracker = ValueTracker(0)
        square.add_updater(
            lambda x: x.become(Polygon(p1, p2*np.array([1, 1-s_tracker.get_value(), 1]), p2*np.array([1, 1-s_tracker.get_value(), 1])-np.array([0, 2*r, 0]), p4,
                                color=colour_quad, **common_kwargs))
        )
        self.play(s_tracker.animate.increment_value(1), run_time=rt)
        text_pc = MathTex("pc").shift(square.get_center())
        self.play(FadeIn(text_pc), FadeIn(text_eq))

## Exponentielles Wachstum

Es gibt eine berühmte Aufgabenart zu exponentiellem Wachstum, bei der man annimmt, dass eine Seerose ihre Fläche in einem Teich täglich verdoppelt.
Die folgende Veranschaulichung ist nicht sehr hübsch und die Animationen zwischen den Tagen sind nicht wirklich exponentiell. Hier könnte man viel verbessern...

In [6]:
%%manim -ql -v WARNING Teich

common_kwargs = {"fill_opacity": 0.9}

class Teich(Scene):
    step_num = 3
    width = 0.5
    height = 0.5/np.sqrt(2)
    pondw = 2**step_num*width
    pondh = 2**step_num*height


    def construct(self):
        day = Tex("Tag: ")
        daynum = Integer(0).next_to(day).align_to(day, UP)
        theday = VGroup(day, daynum).shift(self.pondh*DOWN)
        rose = Rectangle(width=self.width, height=self.height, color=GREEN, **common_kwargs).shift(
            (self.pondw - self.width)/2*LEFT + (self.pondh - self.height)/2*DOWN
        )
        pond = Rectangle(width=self.pondw, height=self.pondh, color=BLUE, **common_kwargs)
        self.play(FadeIn(pond), FadeIn(rose), FadeIn(theday))

        for x in range(self.step_num):
            newrose = rose.copy()
            self.add(newrose)
            self.play(newrose.animate.shift(self.height*UP))
            daynum.set_value(2*x+1)
            self.height *= 2
            rose = VGroup(rose, newrose)
            newrose = rose.copy()
            self.add(newrose)
            self.play(newrose.animate.shift(self.width*RIGHT))
            daynum.set_value(2*x+2)
            self.width *= 2
            rose = VGroup(rose, newrose)

## Newtonverfahren

In [8]:
%%manim -ql -v WARNING Newton

class Newton(Scene):
    x0 = 0.5
    xs = [x0]
    lines = []
    step_num = 3

    def p(self, x):
        return x**2 - 2

    def pprime(self, x):
        return 2*x

    def plot_line(self, ax):
        x0 = self.xs[-1]
        return ax.plot(lambda x: self.p(x0) + (x-x0)*self.pprime(x0), color="green")
    
    def new_x(self):
        x0 = self.xs[-1]
        return x0 - self.p(x0)/self.pprime(x0)

    def construct(self):
        ax = Axes(
            y_range=[-3, 2.5, 1],
            x_range=[-0.5, 2.5, 1]
        ).add_coordinates()
        ax_labels = ax.get_axis_labels()

        graph = ax.plot(lambda x: self.p(x), x_range=[-2.5, 2.5], color="blue")
        self.play(Create(ax), FadeIn(ax_labels))
        #self.play(Create(ax_labels))
        self.play(Create(graph))

        x0 = self.xs[0]
        
        for step in range(self.step_num):
            vert_line = ax.plot_line_graph([x0, x0], [0, self.p(x0)], add_vertex_dots=False)
            self.play(Create(vert_line))
            new_line = self.plot_line(ax)
            self.play(Create(new_line))
        
            x0 = self.new_x()
            self.xs.append(x0)